## Code for performing inference with ViFi-CLIP on custom videos

### Please set the corresponding values in the cell below. Afterwards, just run the cells for inference with ViFi-CLIP model

In [11]:
### Set values here ###
config = 'configs/zero_shot/train/k400/16_16_vifi_clip.yaml'
output_folder_name = "exp"
pretrained_model_path = "/home/gamerx/Downloads/vifi_clip_10_epochs_k400_full_finetuned.pth"
# List the action names for which ViFi-CLIP will perform action recognition
class_names = ['dancing', 'drum beating', 'swimming', "climbing stairs"]
# Load your video example:
video_path = '/home/gamerx/Downloads/hmdb51_org/climb_stairs/Walking_up_the_stairs_climb_stairs_l_cm_np1_ba_med_1.avi'

### Import libraries 

In [2]:
import torch
import torch.nn as nn
from utils.config import get_config
from utils.logger import create_logger
import time
import numpy as np
from utils.config import get_config
from trainers import vificlip
from datasets.pipeline import *

/home/gamerx/anaconda3/envs/vclip/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/gamerx/Documents/PhD_research/clean_repos/ViFi-CLIP/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")
/home/gamerx/anaconda3/envs/vclip/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


### Setting up configuration, no need to change anything.

In [3]:
# Step 1:
# Configuration class 
class parse_option():
    def __init__(self):
        self.config = config
        self.output =  output_folder_name   # Name of output folder to store logs and save weights
        self.resume = pretrained_model_path
        # No need to change below args.
        self.only_test = True
        self.opts = None
        self.batch_size = None
        self.pretrained = None
        self.accumulation_steps = None
        self.local_rank = 0
args = parse_option()
config = get_config(args)
# logger
logger = create_logger(output_dir=args.output, name=f"{config.MODEL.ARCH}")
logger.info(f"working dir: {config.OUTPUT}")

=> merge config from configs/zero_shot/train/k400/16_16_vifi_clip.yaml
[2023-11-24 23:42:45 ViT-B/16](588939814.py 19): INFO working dir: exp


### Loading ViFi-CLIP and its pretrained weights

In [4]:
# Step 2:
# Create the ViFi-CLIP models and load pretrained weights
model = vificlip.returnCLIP(config,
                            logger=logger,
                            class_names=class_names,)
model = model.float().cuda()  # changing to cuda here


[2023-11-24 23:42:45 ViT-B/16](vificlip.py 202): INFO Loading CLIP (backbone: ViT-B/16)
[2023-11-24 23:42:49 ViT-B/16](vificlip.py 205): INFO Building ViFi-CLIP CLIP
[2023-11-24 23:42:49 ViT-B/16](vificlip.py 222): INFO Turning on gradients for COMPLETE ViFi-CLIP model
[2023-11-24 23:42:49 ViT-B/16](vificlip.py 245): INFO Parameters to be updated: {'text_encoder.transformer.resblocks.1.mlp.c_proj.bias', 'image_encoder.transformer.resblocks.1.mlp.c_fc.bias', 'text_encoder.transformer.resblocks.1.attn.out_proj.weight', 'text_encoder.transformer.resblocks.4.attn.in_proj_weight', 'image_encoder.transformer.resblocks.2.attn.in_proj_bias', 'image_encoder.transformer.resblocks.3.ln_1.weight', 'text_encoder.transformer.resblocks.11.attn.in_proj_bias', 'image_encoder.transformer.resblocks.3.attn.in_proj_bias', 'image_encoder.transformer.resblocks.10.ln_1.weight', 'image_encoder.transformer.resblocks.6.mlp.c_fc.weight', 'image_encoder.transformer.resblocks.7.mlp.c_fc.bias', 'image_encoder.transf

In [5]:
logger.info(f"==============> Resuming form {config.MODEL.RESUME}....................")
checkpoint = torch.load(config.MODEL.RESUME, map_location='cpu')
load_state_dict = checkpoint['model']
# now remove the unwanted keys:
if "module.prompt_learner.token_prefix" in load_state_dict:
    del load_state_dict["module.prompt_learner.token_prefix"]

if "module.prompt_learner.token_suffix" in load_state_dict:
    del load_state_dict["module.prompt_learner.token_suffix"]

if "module.prompt_learner.complete_text_embeddings" in load_state_dict:
    del load_state_dict["module.prompt_learner.complete_text_embeddings"]
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in load_state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

[2023-11-24 23:42:52 ViT-B/16](2665388570.py 1): INFO ==============> Resuming form /home/gamerx/Downloads/vifi_clip_10_epochs_k400_full_finetuned.pth....................


In [6]:
# load params
msg = model.load_state_dict(new_state_dict, strict=False)
logger.info(f"resume model: {msg}")

[2023-11-24 23:42:53 ViT-B/16](3852643250.py 3): INFO resume model: _IncompatibleKeys(missing_keys=['prompt_learner.complete_text_embeddings'], unexpected_keys=[])


### Preprocessing input video 

In [7]:
# Step 3: 
# Preprocessing for video
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_bgr=False)
scale_resize = int(256 / 224 * config.DATA.INPUT_SIZE)
val_pipeline = [
    dict(type='DecordInit'),
    dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=config.DATA.NUM_FRAMES, test_mode=True),
    dict(type='DecordDecode'),
    dict(type='Resize', scale=(-1, scale_resize)),
    dict(type='CenterCrop', crop_size=config.DATA.INPUT_SIZE),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='FormatShape', input_format='NCHW'),
    dict(type='Collect', keys=['imgs'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs'])
]
if config.TEST.NUM_CROP == 3:
    val_pipeline[3] = dict(type='Resize', scale=(-1, config.DATA.INPUT_SIZE))
    val_pipeline[4] = dict(type='ThreeCrop', crop_size=config.DATA.INPUT_SIZE)
if config.TEST.NUM_CLIP > 1:
    val_pipeline[1] = dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=config.DATA.NUM_FRAMES, multiview=config.TEST.NUM_CLIP)
pipeline = Compose(val_pipeline)

In [8]:
dict_file = {'filename': video_path, 'tar': False, 'modality': 'RGB', 'start_index': 0}

### ViFi-CLIP inference with given video

In [9]:
video = pipeline(dict_file)
video_tensor = video['imgs'].unsqueeze(0).cuda().float()
# Inference through ViFi-CLIP
with torch.no_grad():
    with torch.cuda.amp.autocast():
        logits = model(video_tensor)
pred_index = logits.argmax(-1)

In [10]:
print(f'logits: {logits}')
print(f'predicted action category is : {class_names[pred_index]}')

logits: tensor([[12.5938,  9.1016, 11.2266, 14.6953]], device='cuda:0',
       dtype=torch.float16)
predicted action category is : climbing stairs
